In [17]:
import os
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
#local os인 경우와 colap 인 경우 디렉토리 설정은 각각 다음과 같음
imdb_dir ='aclImdb'
#imdb_dir = '/content/drive/MyDrive/Colab Notebooks/dataset/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')
labels = []
texts = []

In [18]:
# 데이터 로드 및 라벨링
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    print(dir_name) # 확인용 출력
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            with open(os.path.join(dir_name, fname), encoding='utf8') as f:
                texts.append(f.read())
            labels.append(0 if label_type == 'neg' else 1)

print(texts[0]) # 부정의 첫 번째 문장
print(labels[0]) # 부정의 첫 번째 Label
print(texts[12500]) # 긍정의 첫 번째 문장
print(labels[12500]) # 긍정의 첫 번째 Label

aclImdb/train/neg
aclImdb/train/pos
Working with one of the best Shakespeare sources, this film manages to be creditable to it's source, whilst still appealing to a wider audience.<br /><br />Branagh steals the film from under Fishburne's nose, and there's a talented cast on good form.
0
For a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. The Moroni character is an absolute scream. Watch for Alan "The Skipper" Hale jr. as a police Sgt.
1


In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np 
from tensorflow.keras.models import Sequential
from tensorflow.keras. layers import Embedding, LSTM, Dense, Bidirectional, Dropout 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# 데이터 전처리 설정
maxlen = 200 # 200개 단어 이후는 버립니다
training_samples = 10000
validation_samples = 10000
max_words = 10000 # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다

# 토큰화 및 패딩
tokenizer = Tokenizer (num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('%s개의 고유한 토큰을 찾았습니다.' % len(word_index))
data = pad_sequences(sequences, maxlen=maxlen)
labels = np.asarray(labels)
print('데이터 텐서의 크기:'
, data.shape)
print('레이블 텐서의 크기:', labels.shape)

88582개의 고유한 토큰을 찾았습니다.
데이터 텐서의 크기: (25000, 200)
레이블 텐서의 크기: (25000,)


In [20]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

# 모델 정의
model = Sequential()
model.add(Embedding(max_words, 64, input_length=maxlen))
model.add (Bidirectional(LSTM(64, return_sequences=True)))
model.add (Dropout(0.5))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(1, activation='sigmoid'))

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [23]:
# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')
# 모델 학습
history = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_val, y_val),
                    callbacks=[early_stopping, model_checkpoint])

Epoch 1/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 29s 174ms/step - acc: 0.6211 - loss: 0.6229 - val_acc: 0.8189 - val_loss: 0.4055
Epoch 2/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 38s 241ms/step - acc: 0.8919 - loss: 0.2776 - val_acc: 0.8587 - val_loss: 0.3552
Epoch 3/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 37s 234ms/step - acc: 0.9497 - loss: 0.1476 - val_acc: 0.8430 - val_loss: 0.4589
Epoch 4/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 36s 227ms/step - acc: 0.9683 - loss: 0.0958 - val_acc: 0.8357 - val_loss: 0.5799
Epoch 5/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 36s 229ms/step - acc: 0.9815 - loss: 0.0612 - val_acc: 0.8377 - val_loss: 0.6214
Epoch 6/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 37s 235ms/step - acc: 0.9891 - loss: 0.0400 - val_acc: 0.8379 - val_loss: 0.5948
Epoch 7/20
157/157 ━━━━━━━━━━━━━━━━━━━━ 37s 236ms/step - acc: 0.9912 - loss: 0.0328 - val_acc: 0.8389 - val_loss: 0.6673


In [29]:
# 모델을 사용하여 예측
test_texts = [ #아래 문장은 강의 채널에 올려 놓음
    "I hated this movie. It was terrible and the acting was horrible.",
    "This was the worst film I have ever seen. Not worth the time.",
    "I loved this movie. It was fantastic and the acting was great.",
    "This was the best film I have seen in a long time. Totally worth it.",
    "I had high hopes for this movie, but it was a complete letdown . . . ",
    "This film was a disaster from start to finish. The dialogue  . . . ",
    "What an amazing film! The plot was deeply engaging, . . . ",
    "I was thoroughly impressed by this film. . . . "
]

test_sequences = tokenizer.texts_to_sequences(test_texts)
test_data = pad_sequences(test_sequences, maxlen=maxlen)

from tensorflow.keras.models import load_model
best_model = load_model('best_model.keras', custom_objects=None, compile=True)

predictions = best_model.predict(test_data)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step


In [30]:

for i, test_text in enumerate(test_texts):
    print(f"Text: {test_text[:20]}...")
    print(f"Prediction: {'Positive' if predictions[i] > 0.5 else 'Negative'} (확률: {predictions[i][0]:.4f})")

Text: I hated this movie. ...
Prediction: Negative (확률: 0.0540)
Text: This was the worst f...
Prediction: Negative (확률: 0.1854)
Text: I loved this movie. ...
Prediction: Positive (확률: 0.9273)
Text: This was the best fi...
Prediction: Positive (확률: 0.7730)
Text: I had high hopes for...
Prediction: Negative (확률: 0.2273)
Text: This film was a disa...
Prediction: Negative (확률: 0.3773)
Text: What an amazing film...
Prediction: Negative (확률: 0.3293)
Text: I was thoroughly imp...
Prediction: Positive (확률: 0.5323)
